In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
cd /content/drive/MyDrive/projects/mask_dataset

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/projects/mask_dataset'
c:\Users\DELL\Desktop\microsfot


In [4]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Data Preprocessing

In [6]:
import cv2,os
data_path='/content/drive/MyDrive/projects/mask_dataset'
files = os.listdir(data_path)
categories = []

for file in files:
  if os.path.isdir(file):
    categories.append(file)

labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

ModuleNotFoundError: No module named 'cv2'

In [6]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.

Recale and assign catagorical labels

In [7]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [8]:
new_target.shape

(1376, 2)

In [9]:
np.save('images.npy',data)
np.save('labels.npy',new_target)

CNN Model

In [10]:
!ls

CNN_Model.png	 model-003.model  model-008.model  with_mask
images.npy	 model-004.model  model-011.model  without_mask
labels.npy	 model-005.model  model-012.model
model-001.model  model-006.model  model-016.model
model-002.model  model-007.model  model-017.model


In [11]:
import numpy as np
data=np.load('images.npy')
new_target=np.load('labels.npy')

In [12]:
data.shape

(1376, 100, 100, 1)

In [13]:
data.shape[1:]

(100, 100, 1)

In [14]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 200)       2000      
                                                                 
 activation (Activation)     (None, 98, 98, 200)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 200)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       180100    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 100)      0         
 2D)                                                    

Splittiong data into traning and testing

In [16]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [17]:
train_data.shape

(1238, 100, 100, 1)

In [18]:
train_target.shape

(1238, 2)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=100,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/100
31/31 [==============================] - 92s 3s/step - loss: 0.6944 - accuracy: 0.5545 - val_loss: 0.6049 - val_accuracy: 0.7056
Epoch 2/100
31/31 [==============================] - 90s 3s/step - loss: 0.5468 - accuracy: 0.7394 - val_loss: 0.4708 - val_accuracy: 0.7984
Epoch 3/100
31/31 [==============================] - 90s 3s/step - loss: 0.4102 - accuracy: 0.8242 - val_loss: 0.3075 - val_accuracy: 0.9073
Epoch 4/100
31/31 [==============================] - 90s 3s/step - loss: 0.2868 - accuracy: 0.8758 - val_loss: 0.2062 - val_accuracy: 0.9234
Epoch 5/100
31/31 [==============================] - 90s 3s/step - loss: 0.2269 - accuracy: 0.9131 - val_loss: 0.1819 - val_accuracy: 0.9435
Epoch 6/100
31/31 [==============================] - 90s 3s/step - loss: 0.1627 - accuracy: 0.9404 - val_loss: 0.1434 - val_accuracy: 0.9476
Epoch 7/100
31/31 [==============================] - 89s 3s/step - loss: 0.1407 - accuracy: 0.9535 - val_loss: 0.1230 - val_accuracy: 0.9637
Epoch 8/100
3

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# plot the training loss and accuracy
N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")